## **rgee in Colab**
<a href="https://colab.research.google.com/github/r-spatial/rgee/blob/examples/rgee_colab.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
<br>
OBS: Consider that **you can not** run **`googledrive`** and **`googleCloudStorageR`** in a non-interactive session. The following functions will not work.
- ee_drive_to_local()
- ee_as_raster(via = "drive")
- ee_as_stars(via = "drive")
- ee_as_sf(via = "drive")
- rgee::ee_gcs_to_local()
- ee_as_raster(via = "gcs")
- ee_as_stars(via = "gcs")
- ee_as_sf(via = "gcs")
- sf_as_ee(via = "gcs_to_asset")
- gcs_to_ee_image
- raster_as_ee
- local_to_gcs- stars_as_ee

In [ ]:
#@title **Install rgee dependencies. It takes aprox. 12 min**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('sf')
install.packages('mapview')
install.packages('cptcity')
install.packages('geojsonio')
remotes::install_github("r-spatial/rgee@rgeev.1.0.3")

In [2]:
#@title **help functions**
help <- function(func){
  func <- as.character(substitute(func))
  IRdisplay::display(utils::help(func))
}

ee_help <- function(func){  
  func <- as.character(substitute(func))
  tmp_html <- rgee::ee_help(func)
  IRdisplay::display_html(file = tmp_html)
}

In [3]:
#@title **embed function**
# Created by cromulent
# https://stackoverflow.com/questions/46770320/error-html-widgets-cannot-be-represented-in-plain-text
ee_embed = function(x, height = 450) {    
    tmp = tempfile(fileext = ".html")
    htmlwidgets::saveWidget(x@map, tmp)
    rawHTML = base64enc::dataURI(mime = "text/html;charset=utf-8", file = tmp)
    IRdisplay::display_html(paste("<iframe src=", rawHTML, "width=100% height=", height, "id=","igraph", "scrolling=","no","seamless=","seamless", "frameBorder=","0","></iframe>", sep = "\""))
    unlink(tmp)
}

### **1. Initialize and Authenticate**

In [ ]:
library(rgee)
ee_Initialize(display = TRUE)

### **2. Hello world!**

In [ ]:
library(rgee)
library(sf)
library(cptcity)
library(mapview)

In [ ]:
# Access to documentation through "help" for R functions and "ee_help" for Earth Engine methods
# Psst: Use Ctrl + Shift + Enter to run line by line.
ee_help("ee$Image")
IRdisplay::display_html('<br><br><br>')
help(ee_image_info)

In [ ]:
# Create a color palette
cpt_pal <- cpt(pal = "mpl_inferno")

# Load an EE Image
image <- ee$Image('CGIAR/SRTM90_V4')

# Display results with Map$addLayer!
Map$centerObject(image)
m1 <- Map$addLayer(
  eeObject = image,
  visParams = list(min = 0, max = 5000, palette= cpt_pal),  
  name = 'SRTM90_V4'
)
m1 %>% ee_embed

In [ ]:
# You can also combine Map$addLayer with mapview!
nc <- st_read(system.file("shape/nc.shp", package="sf"))

# Move a sfc from local to Earth Engine
ee_nc <- nc %>%
  st_bbox %>% 
  st_as_sfc %>%
  sf_as_ee   
m2 <- Map$addLayer(ee_nc,list(color="FFFFFF"))

# Putting all together!
(mapview(nc) + m1 + m2) %>% ee_embed

<center>
<img src="https://user-images.githubusercontent.com/16768318/89118214-486d5200-d469-11ea-8e16-085cc1888df6.png" width=40% >
</center>